# Загрузка Pandas и очистка данных

In [363]:
import pandas as pd
import numpy as np
import re
from multiprocessing import Pool
import workers

In [364]:
df = pd.read_csv('main_task.csv')

In [365]:
# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки
df.info(max_cols=300)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [366]:
# При необходимости добавьте ячейки
df.info(max_cols=200)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [367]:
#Т.к. в столбце "Price Range" больше всего пропуском, используем ссылки из пропущщеных значений дял запросов к сайту.
#Получим с сайта Информацию о цене, видах кухни и количестве отзывов, сохраним эту информацию в DataFrame и в файл parse.csv

#if __name__ == '__main__':
#    with Pool(processes=40) as pool:
#        parse_df = pd.DataFrame(pool.map(workers.make_price, df[df['Price Range'].isna()]["URL_TA"]), columns=['Price Range', 'Cuisine Style', 'URL_TA', 'Number of Reviews'])

In [368]:
#Приведём значения и пропуски к нужным нам типам. так же удалим строки в которых значения нужных нам столбцов пусты
parse_df = pd.read_csv('parse.csv', index_col=["Unnamed: 0"])

#parse_df.replace(to_replace = {"Nan": np.nan}, inplace=True)
#parse_df.replace(to_replace = {"nan": np.nan}, inplace=True)
#parse_df[parse_df["Cuisine Style"].notna()]["Cuisine Style"] = parse_df["Cuisine Style"].astype(str)
#parse_df.dropna(subset=['Price Range', 'Cuisine Style', 'Number of Reviews'], how="all", inplace=True)
parse_df.drop_duplicates(subset=['URL_TA'], inplace=True)
#parse_df.to_csv("parse.csv", )

In [369]:
#Создадим промежуточный DataFrame и обновим в нём данные полученными с сайта. Так как 
r = df[df["Price Range"].isna()]
r = r.reset_index().set_index("URL_TA")
r.update(parse_df.set_index("URL_TA"), overwrite=False)
r.set_index("index", inplace=True)

In [371]:
df.loc[r.index, ['Cuisine Style','Price Range', 'Number of Reviews']] = r[['Cuisine Style','Price Range', 'Number of Reviews']]

In [372]:
price_dict = {'$': 1, '$$ - $$$': 2.5, '$$$$': 4}
df['Price Range'].replace(to_replace = price_dict, inplace=True)

In [373]:
df = pd.concat([df, pd.get_dummies(df['City'])], axis = 1)

In [374]:
number_city_residents_dict = {'Paris':2148327, 'Stockholm':961609, 'London':8908081, 'Berlin':3644826, 'Munich':1471508, 'Oporto':237591,
       'Milan':1378689, 'Bratislava':437725, 'Vienna':1897491, 'Rome':2870500, 'Barcelona':1636762, 'Madrid':3266126,
       'Dublin':1173179, 'Brussels':179277, 'Zurich':428737, 'Warsaw':1790658, 'Budapest':1752286, 'Copenhagen':615993,
       'Amsterdam':872757, 'Lyon':506615, 'Hamburg':1841179, 'Lisbon':505526, 'Prague':1301132, 'Oslo':673469,
       'Helsinki':655281, 'Edinburgh':488100, 'Geneva':200548, 'Ljubljana':284355, 'Athens':664046,
       'Luxembourg':602005, 'Krakow':769498}
# словарь с кол-вом жителей
df['population'] = df['City'].replace(number_city_residents_dict)

In [375]:
df['rank_on_pop'] = df['Ranking']/df['population']

In [376]:
df['NAN_Cuisine Style'] = pd.isna(df['Cuisine Style']).astype('float64') 
cuisine_count = {}
for cuisines in df['Cuisine Style'].value_counts().index:
    for cuisine in cuisines.strip("[]").replace("'", "").split(', '):
        if cuisine in cuisine_count:
            cuisine_count[cuisine] += 1
        else:
            cuisine_count[cuisine] = 1

df['Cuisine Style'] = df['Cuisine Style'].fillna("['Other']")

df['Cuisine Style'] = df['Cuisine Style'].apply(lambda x: len(x.split(',')))

def find_item(cell):
    if type(cell) == str:
        if item in cell:
            return 1
    return 0

for item in cuisine_count.keys():

    df[item] = df['Cuisine Style'].apply(find_item)

In [377]:
sum(cuisine_count.values())/len(cuisine_count)

286.1418918918919

In [378]:
len([cuisine for cuisine in cuisine_count.keys() if cuisine_count[cuisine] < 286])

110

In [379]:
#Заполним пропуски самым частым значением
df['NAN_Number of Reviews'] = pd.isna(df['Number of Reviews']).astype('float64') 
df['Number of Reviews'].fillna(df['Number of Reviews'].mode()[0], inplace = True)
df['NAN_Price Range'] = pd.isna(df['Price Range']).astype('float64') 
df['Price Range'].fillna(df['Price Range'].mode()[0], inplace = True)

In [380]:
#ВЫделим дату первого и второго отзыва. и заполним пропуски самым частым значенем в каждом столбце
def reviews_date(rew, count):
    date = re.findall(r'\d\d?/\d\d?/\d+', rew)
    #print(date, len(date), len(data) == 2)
    if len(date) == 0:
        return np.nan
    if count == 1:
        return pd.to_datetime(date[0])
    elif len(date) == 2:
        return pd.to_datetime(date[1])
    else:
        return np.nan

df['Reviews_first'] = df['Reviews'].apply(lambda x: reviews_date(x, 1))
df['Reviews_second'] = df['Reviews'].apply(lambda x: reviews_date(x, 2))
df['Reviews_first'].fillna(df['Reviews_first'].mode()[0], inplace = True)
df['Reviews_second'].fillna(df['Reviews_second'].mode()[0], inplace = True)

In [381]:
#добавим столбец содержащий количество дней между первым и вторым отзывом
df['Reviews_between'] = (df['Reviews_first'] - df['Reviews_second']).dt.days

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [382]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['City', 'Reviews', 'URL_TA', 'ID_TA', 'Reviews_first', 'Reviews_second', 'Restaurant_id', 'Rating'], axis = 1)
y = df['Rating']

In [383]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [384]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [385]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [386]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [387]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.2141785


In [388]:
0.21171399999999999
0.213905
0.2133565
0.21309899999999998

0.21309899999999998